In [50]:
import cartopy.crs as ccrs
from datetime import datetime, timedelta
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import xarray as xr
import shutil

In [51]:
def one_more_day(start):
    "Given a date in the format YYYY-MM-DD it adds 1 day and return it in the same format"
    # convert into a datetime object
    start_date =datetime.strptime(start, '%Y-%m-%d')

    # find the next day 
    end_date = start_date +timedelta(days=1)

    #convert into a string
    end_fire = end_date.strftime('%Y-%m-%d')
    
    return end_fire

def select_netCDF(path, date):
    """
    Given a date, selects the corresponding NetCDF file ending with *YYYYMM.nc.
    """
    year, month, _ = date.split('-')  # Extract year and month from the date string

    # Search for files ending with *YYYYMM.nc
    files = [
        os.path.join(dirpath, filename)
        for dirpath, _, filenames in os.walk(path)
        for filename in filenames if filename.endswith(f'{year}{month}.nc')
    ]
    if not files:  # If the list is empty, return None or handle the error appropriately
        return None
    return files[0]

def calculate_square_bounds(lon, lat, side_km):
    """
    Calculate the bounds of a square centered at a given lon/lat.
    
    Parameters:
        lon (float): Longitude of the center.
        lat (float): Latitude of the center.
        side_length_km (float): Side length of the square in kilometers.
    
    Returns:
        dict: Dictionary with the extreme coordinates of the square.
    """
    earth_circ_km = 40075 #km
    km_per_degree =  earth_circ_km/360  # this gets how much is a degree in the equator


    # Calculate how much our sqaure will be 
    half_side_lat_degree = (side_km / 2) / (km_per_degree) 
    half_side_lon_degree = (side_km / 2) / (km_per_degree * np.cos(np.deg2rad(lat))) # this accounts for the fact that 
                                                                                     # meridians are closer together as 
                                                                                     # you approach the poles
    
    # Calculate bounds
    lat_min = lat - half_side_lat_degree
    lat_max = lat + half_side_lat_degree
    lon_min = lon - half_side_lon_degree
    lon_max = lon + half_side_lon_degree

    # Return the bounds as a tuple
    return lat_min, lat_max, lon_min, lon_max

def select_area_climex(wind_data, lat_min, lat_max, lon_min, lon_max):
    
    # load the rotated coordinates system
    pole_longitude = wind_data.rotated_pole.grid_north_pole_longitude
    pole_latitude = wind_data.rotated_pole.grid_north_pole_latitude
    rotated_pole = ccrs.RotatedPole(pole_longitude=pole_longitude, pole_latitude=pole_latitude)

    # transform the lon/lat min/max into the rotated coordinates
    geodetic = ccrs.PlateCarree()

    # Convert geographic lat/lon to rotated coordinates
    rlon_min, rlat_min = rotated_pole.transform_point(lon_min, lat_min, geodetic)
    rlon_max, rlat_max = rotated_pole.transform_point(lon_max, lat_max, geodetic)


    # Select the area
    region = wind_data.sel(rlon=slice(rlon_min, rlon_max), rlat=slice(rlat_min, rlat_max))

    return region

def create_wind_dataframe_era5(start_fire, region, lon, lat, duration=12, step=3):
    """
    Create a DataFrame with avearge wind data for 18h is stepts of 3h for the region
    
    Parameters:
    - start_fire (str): Start date of the fire in 'YYYY-MM-DD' format.
    - region (xarray.Dataset): Dataset containing wind data with 'wind_speed' and 'wind_direction' variables.
    - lon (float): Longitude of the fire location.
    - lat (float): Latitude of the fire location.
    - duration (int): Total duration in hours to consider (default: 18 h).
    - step (int): Time step interval in hours (default: 3).
    
    Returns:
    - pd.DataFrame: DataFrame with columns ['lon', 'lat', 'time_step', 'wind_speed', 'wind_direction'].
    """

    # Calculate the number of time steps
    time_steps = int(duration / step)

    # initialize dataframe
    df = pd.DataFrame(columns=['date', 'wind_speed', 'wind_direction', 'lon', 'lat'])

    # loop through each time step
    for i in range(time_steps):
        # Calculate the time_step
        if i == 0:
            time_step = datetime.strptime(start_fire, '%Y-%m-%d') + timedelta(hours=12)
        else:
            time_step = datetime.strptime(start_fire, '%Y-%m-%d') + timedelta(hours=12) + i * timedelta(hours=step)
        
        # Calculate the previous time step for averaging
        previous_time_step = time_step - timedelta(hours=2) # 2h are done because we take 10,11 and 12 (three hours!!!)
        
        # Extract wind data from the region (average over time, lon, and lat)
        wind_data = region.sel(time=time_step).mean(dim=['lon', 'lat'])
        wind_speed = wind_data['wind_speed'].values
        wind_direction = wind_data['wind_direction'].values

        # Format the date as 'YYYY-MM-DD_HH:mm:ssZ'
        formatted_date = time_step.strftime('%Y-%m-%d_%H:%M:%SZ')
   
        #  Create a new row for this time step
        new_row = pd.DataFrame({
            'date': [formatted_date],
            'wind_speed': [wind_speed],
            'wind_direction': [wind_direction],
            'lon': [lon],
            'lat': [lat]
        })

        # Append the new row to the DataFrame
        df = pd.concat([df, new_row], ignore_index=True)

    return df


def create_wind_dataframe_climex(start_fire, region, lon, lat, duration=12, step=3):
    """
    Create a DataFrame with average wind data for a specified duration with steps of 3 hours 
    from the CLIMEX dataset, using the rotated coordinate system.

    Parameters:
    - start_fire (str): Start date of the fire in 'YYYY-MM-DD' format.
    - region (xarray.Dataset): CLIMEX dataset containing wind data in rotated coordinates.
    - lon (float): Longitude of the fire location.
    - lat (float): Latitude of the fire location.
    - duration (int): Total duration in hours to consider (default: 12 h).
    - step (int): Time step interval in hours (default: 3).
    
    Returns:
    - pd.DataFrame: DataFrame with columns ['date', 'wind_speed', 'wind_direction', 'rlon', 'rlat'].
    """

    # Load the rotated coordinate system from CLIMEX dataset
    pole_longitude = region.rotated_pole.grid_north_pole_longitude
    pole_latitude = region.rotated_pole.grid_north_pole_latitude
    rotated_pole = ccrs.RotatedPole(pole_longitude=pole_longitude, pole_latitude=pole_latitude)

    # Convert input geographic coordinates (lon, lat) to rotated coordinates (rlon, rlat)
    geodetic = ccrs.PlateCarree()
    rlon, rlat = rotated_pole.transform_point(lon, lat, geodetic)

    # Calculate the number of time steps
    time_steps = int(duration / step)

    # Initialize the DataFrame
    df = pd.DataFrame(columns=['date', 'wind_speed', 'wind_direction', 'lon', 'lat'])

    # Loop through each time step
    for i in range(time_steps):
        # Calculate the time_step
        if i == 0:
            time_step = datetime.strptime(start_fire, '%Y-%m-%d') + timedelta(hours=12)
        else:
            time_step = datetime.strptime(start_fire, '%Y-%m-%d') + timedelta(hours=12) + i * timedelta(hours=step)
        

        # Extract wind data from the CLIMEX dataset
        wind_data = region.sel(time=np.datetime64(time_step))
        wind_data_clean = wind_data.drop_vars("rotated_pole")  # Remove non-numeric data
        wind_data_mean = wind_data_clean.mean(dim=['rlon', 'rlat'])


        wind_speed = wind_data_mean['wind_speed'].values
        wind_direction = wind_data_mean['wind_direction'].values


        # Format the date as 'YYYY-MM-DD_HH:mm:ssZ'
        formatted_date = time_step.strftime('%Y-%m-%d_%H:%M:%SZ')

        # Create a new row for this time step
        new_row = pd.DataFrame({
            'date': [formatted_date],
            'wind_speed': [wind_speed],
            'wind_direction': [wind_direction],
            'lon': [lon],
            'lat': [lat]
        })

        # Append the new row to the DataFrame
        df = pd.concat([df, new_row], ignore_index=True)
    return df

In [52]:
# Load the fires
pathFires = r"O:\Climate-and-Energy-Policy\CERM\Projects\Wildfire\Data\ML-data-Europe-gridded\results\ignition-selection\fire-season-ignition-points-1992-2010thresh07_climex.csv"
fires = pd.read_csv(pathFires)
fires

,Unnamed: 0,date,lon,lat
0,0,1992-06-01,16.573744,40.100482
1,1,1992-06-01,21.731862,37.372360
2,2,1992-06-01,11.269389,43.851956
3,3,1992-06-01,7.747140,43.805810
4,4,1992-06-01,8.819791,40.539318
...,...,...,...,...
10895,10895,2010-10-27,20.660281,39.902497
10896,10896,2010-10-28,23.368362,37.623702
10897,10897,2010-10-29,23.411173,38.054088
10898,10898,2010-10-30,14.684685,40.671987


In [54]:
climate_data = 'climex2'

if climate_data=='climex2':
    directory = r'O:\Climate-and-Energy-Policy\CERM\Projects\Wildfire\Data\CLIMEX2\wind\original\epp'
elif climate_data == 'era5-land':
    directory =r'O:\Climate-and-Energy-Policy\CERM\Projects\Wildfire\Data\ERA5-land\original\wspd10'

# loop over each row
for index, row in fires.iterrows():
    
    # Store the day and the location of the fire
    date = row['date']
    lon = row['lon']
    lat = row['lat']

    # Create a folder ----------------------------------------
    folfer_path = r"C:\Users\jsoma\Desktop\fires_campania_climex2"
    folder_name =  folfer_path + f"\\{date}_{lon:.2f}E_{lat:.2f}N"
    os.makedirs(folder_name, exist_ok=True)

    # --------------------------------------------------------

    
    # 1) select the proper time to later select the netcdf
    start_fire_date = date
    end_fire_date = one_more_day(start_fire_date)

    file_start_fire = select_netCDF(directory, start_fire_date)
    file_end_fire = select_netCDF(directory, end_fire_date)
    if not file_start_fire:  # Skip the loop if no file is found for start fire date
        if os.path.exists(folder_name):  # Check if the folder exists before removing
            shutil.rmtree(folder_name)  # Remove the folder and its contents
        continue

    else:
        # Open the netCDF files
        wspd10_start = xr.open_dataset(file_start_fire)
        wspd10_start = wspd10_start.sel(time=start_fire_date)

        wspd10_end = xr.open_dataset(file_end_fire)
        wspd10_end = wspd10_end.sel(time=end_fire_date)

        # Concatenate the data along the time dimension
        wind_data = wspd10_start

        # 2) Select the area of the bounds
        side_length_km = 30  # Side length of the square in kilometers
        lat_min, lat_max, lon_min, lon_max = calculate_square_bounds(lon, lat, side_length_km)

        # Open the netCDF files
        wspd10_start = xr.open_dataset(file_start_fire)
        wspd10_start = wspd10_start.sel(time=start_fire_date)

        wspd10_end = xr.open_dataset(file_end_fire)
        wspd10_end = wspd10_end.sel(time=end_fire_date)

        # Concatenate the data along the time dimension
        wind_data = wspd10_start

        # 2) Select the area of the bounds
        side_length_km = 30  # Side length of the square in kilometers
        lat_min, lat_max, lon_min, lon_max = calculate_square_bounds(lon, lat, side_length_km)

        if climate_data == 'climex2':
            region = select_area_climex(wind_data, lat_min, lat_max, lon_min, lon_max)
        else:
            region = wind_data.sel(lat=slice(lat_max, lat_min), lon=slice(lon_min, lon_max))

        # 3) Calculate the average wind profile for the hours the fire runs (18h)
        if climate_data == 'climex2':
            wind_df = create_wind_dataframe_climex(start_fire_date, wind_data, lon, lat)
        else:
            wind_df = create_wind_dataframe_era5(start_fire_date, wind_data, lon, lat)

        # Save the wind data in the folder
        csv_path = os.path.join(folder_name, 'wind_data.csv')
        wind_df.to_csv(csv_path, index=False)
        



C:\Users\jsoma\AppData\Local\Temp\ipykernel_4280\2050794838.py:201: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_row], ignore_index=True)
